# API authentication using HMAC, SHA256 and BASE64

Sometimes APIs are given access through the use of authentication tokens. For example in the case of the customer feedback API of [Mopinion](https://mopinion.com/), we need to create an authentication token of the following pattern:

```
BASE64( PUBLIC_KEY : HMAC.SHA256( RESOURCE_URI | JSON_BODY )
```

> The authentication token is a  Base64 encoded string, consisting of your public key, paired with an  HMAC-signature. The HMAC-signature is unique for each request and must be created following the pattern below.

This notebook demonstrates how to create the above authentication token with Python. 

We will do demonstrate this by recreating the example listed in the [API documentation](https://developer.mopinion.com/api/#section/Authentication).

In [1]:
expected_hmac_signature = "8d0d693f34c58c60fc46cf3a725642dc5a43594bcc8be2537e81c904ee4dc2c1"
expected_auth_token = "MTIzYWJjOjhkMGQ2OTNmMzRjNThjNjBmYzQ2Y2YzYTcyNTY0MmRjNWE0MzU5NGJjYzhiZTI1MzdlODFjOTA0ZWU0ZGMyYzE="

## Step 1:  Create the HMAC key

> The HMAC signature must be created using SHA256-hashing, and encrypted with your signature token.

This can be done with the below code. Note:
* Adding the seperators argument in the `json.dumps()` call avoids adding white spaces.
* `hmac.new()` needs a bytes or bytearray object. Hence the `bytes()` function call.
* The SHA256-hashing is provided through the `digestmod=hashlib.sha256` argument.
* `.hexdigest()` returns the signature in the required hexadecimal format.

In [2]:
import hmac
import hashlib
import json

# example data
signature_token = "$1gn47ur3T0k3n"
resource_uri = '/reports'
json_body = {
    "name":"My new report",
    "language":"en"
}

# create payload
json_string = json.dumps(json_body, separators=(',', ':'))
message = '{}|{}'.format(resource_uri, json_string)

# create HMAC-signature
hmac_signature = hmac.new(
    bytes(signature_token, 'ascii'), 
    msg = bytes(message, 'ascii'), 
    digestmod = hashlib.sha256
).hexdigest()

# validate if correct
print(hmac_signature == expected_hmac_signature)

True


## Step 2:  Create the authentication token

In a second step we need to combine the public key with the above created HMAC-signature in a Base64 encoded string. 

This can be done using the `base64` library:

In [3]:
import base64

# example data
public_key = '123abc'

# payload
message = '{}:{}'.format(public_key, hmac_signature)

# create authentication token
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
auth_token = base64_bytes.decode('ascii')

# validate if correct
print(auth_token == expected_auth_token)

True


That's it!

Done.